In [41]:
f = open("log.txt", "r")
fout = open("domain_report", "w")
g = open("suspicious_report", "w")
gout = open("irresponsible_behavior","w")
h = open("glitch_report", "w")
def byDate(line):
    # get the date and sort by it
    return line.split()[0]
def byTime(line):
    # get the time and sort by it
    return line.split()[1]
logstmts = []
for line in f:
    logstmts.append(line)
logstmts.sort(key=byDate)

# {
# 	['bob@gmail.com'] : {
# 							['2020-05-15'] : [
# 										'2020-05-15 10:54:11      		logout    		mailserver.local         		bob@miho-nakayama.com',
# 										'2020-05-15 10:54:11      		logout    		mailserver.local         		bob@miho-nakayama.com',
# 										'2020-05-15 10:54:11      		logout    		mailserver.local         		bob@miho-nakayama.com',
# 										],
# 							['2020-05-16'] : [
# 										'2020-05-16 10:54:11      		logout    		mailserver.local         		bob@miho-nakayama.com',
# 										'2020-05-16 10:54:11      		logout    		mailserver.local         		bob@miho-nakayama.com',
# 										]
# 						},
# }

# user_dict
# 		k: date => line.split()[0]
# 		v: [line1, line2, ...]
user_dict = {}
for line in logstmts:
    current_date = line.split()[0]
    email = line.split()[4]
    try:
        email_dict = user_dict[email]
        try:
            email_dict[current_date].append(line)
        except:
            # need to insert date
            email_dict[current_date] = [line]        
    except:
        # need to insert email with value dict
        user_dict[email] = {current_date:[line]}
# sort the lines by time in every date dict
for user in user_dict:
    for date in user_dict[user]:
        user_dict[user][date].sort(key=byTime)
behaviors = {}


def increment_behavior(user):
    try: 
        behaviors[user] += 1 
    except:
        behaviors[user] = 1

        
def count_illegal_behaviors():
    for user in user_dict:
        sorted_dates = sorted(user_dict[user].keys())
        for date in sorted_dates:
            # num of logins for a day is more than 5, log it
            if len(user_dict[user][date]) > 5:
                increment_behavior(user)
            for line in user_dict[user][date]:
                time = line.split()[1]
                # illegal time
                if 0 <= int(time.split(':')[0]) <= 5:
                    increment_behavior(user)
    g.write("%s " % behaviors)
    
    
# this is used for both p2 and p3, as the only thing that changes is comparison of logouts/logins
def count_logout_behaviors(mode):
    for user in user_dict:
        sorted_dates = sorted(user_dict[user].keys())
        for date in sorted_dates:
            login_count, logout_count = (0, 0)
            for line in user_dict[user][date]:
                # count num of logins vs logouts
                if line.split()[2] == "login":
                    login_count += 1
                else: 
                    logout_count += 1
            if mode == "glitch" and login_count < logout_count:
                increment_behavior(user)
            elif mode != "glitch" and login_count > logout_count: # same as above
                increment_behavior(user)
    h.write("%s" % login_count)
    h.write("%s" % logout_count)
                
                
def count_domains():
    domains = {}
    for user in user_dict:
        try: 
            user_split = user.split('@')
            domains[user_split[1]].add(user_split[0])
        except:
            domains[user_split[1]] = set()
            domains[user_split[1]].add(user_split[0])
    fout.write("==============================\n")
    fout.write("=== DOMAIN COUNT (%s DOMAINS) ===\n" % (len(domains.keys())) )
    fout.write("==============================\n")
    for domain in domains:
        fout.write("%s\t\t\t\t%s\n" % (domain, len(domains[domain])))


        
# write the dictionary with the behavior name
def write_behaviors(behavior_name):
    
    gout.write("==============================\n")
    gout.write("=== %s (%s cases) ===\n" % (behavior_name, len(behaviors.keys())) )
    gout.write("==============================\n")
    
    for user in behaviors:
        gout.write("%s\t\t\t\t%s\n" % (user, behaviors[user]))
        for date in user_dict[user]:
            gout.write("\tDATE: [%s] ---\n" % date)
            for line in user_dict[user][date]:
                split_line = line.split()
                gout.write("\t\t%s\t\t%s\t\t%s\n" % (split_line[1], split_line[2], split_line[3]) )

count_domains()
count_illegal_behaviors()
write_behaviors("Irresponsible Behavior")
count_logout_behaviors("glitch")
gout.close()
f.close()
g.close()